In [17]:
import os
import requests as rq
import pandas as pd
import numpy as np
import json
import re
from bs4 import BeautifulSoup
from tqdm import tqdm,trange
from PIL import Image

In [18]:
Jump = [] # 只有第一次要跑

In [19]:
cinema = {
    "cinemaId":[],  # A-Z
    "cinemaAddress":[], 
    "cinemaName":[], 
    "cinemaTele":[],
    "cinemaImg":[],
    "cinemaBusTwo":[],
    "cinemaInfo":[],
    "cinemaGoogle":[]  # iframe use in html tag
}
root = 'https://www.vscinemas.com.tw/vsweb'  # get img url
path = r"C:\Users\User\Desktop\Cinema_Img"   # yourself u want save
num = 0 # total cinema eq 17

In [20]:
for i in tqdm(range (0,100)):
    if i in Jump:
        continue
    resp = rq.get(
        f"https://www.vscinemas.com.tw/vsweb/theater/detail.aspx?id={i}"
    )
    content = BeautifulSoup(resp.text, "html.parser")
    try:
        cinemaTele = content.select_one('div.theaterPosition article.article div.leftBox ul li.icon-phone p').text
        cinemaAddress = content.select_one('div.theaterPosition article.article div.leftBox ul li.icon-marker p').text
        # 交通資訊大家格式都不太一樣，有的有全形的-，有的不是 所以換兩次
        cinemaBusTwo = content.select_one('div.theaterPosition article.article div.leftBox ul li.icon-bus-two p').text.replace(" －",'\n').replace('－',"").replace("-","")
        cinemaGoogle = content.select_one('div.theaterPosition article.article section.rightMap iframe').get('src')
        # 相關資訊也是，每座的影廳都有放不同位置
        cinemaInfo = content.select('div.theaterStory article.article div.bbsArticle p')[0].text
        if cinemaInfo =='\n'or cinemaInfo =='\xa0' or cinemaInfo =='':
            cinemaInfo = content.select('div.theaterStory article.article div.bbsArticle p')[1].text
            
        cinemaName = content.select('section.crumbArea div.crumb span')[2].text
        cinemaName = re.sub('[a-zA-Z]','',cinemaName).replace(" ","")
        Img = content.select_one('section.theaterBanner img').get('src').replace("..",root)
        Img = rq.get(Img)
        if not os.path.exists(path):
            os.mkdir(path)
        with open(f"{path}\_{cinemaName}.jpg","wb") as file:
            file.write(Img.content)
            Img = Image.open(f"{path}\_{cinemaName}.jpg")
            cinemaImg = f"{path}\_{cinemaName}.jpg"
        cinema['cinemaAddress'].append(cinemaAddress)
        cinema['cinemaBusTwo'].append(cinemaBusTwo)
        cinema['cinemaGoogle'].append(cinemaGoogle)
        cinema['cinemaId'].append(chr(i+64))  # 1 = A so on....台中太魯閣最後一個為26=Z
        cinema['cinemaImg'].append(cinemaImg)
        cinema['cinemaInfo'].append(cinemaInfo)
        cinema['cinemaName'].append(cinemaName)
        cinema['cinemaTele'].append(cinemaTele)
        num += 1
        if num == 17:
            break
    except:
        Jump.append(i)
        print("Not found：",i)
print("Ending")

  1%|▊                                                                                 | 1/100 [00:00<00:53,  1.84it/s]

Not found： 0


  4%|███▎                                                                              | 4/100 [00:08<03:01,  1.89s/it]

Not found： 3


  7%|█████▋                                                                            | 7/100 [00:14<02:42,  1.75s/it]

...

Not found： 23


 25%|████████████████████▎                                                            | 25/100 [00:52<01:55,  1.54s/it]

Not found： 24


 26%|█████████████████████                                                            | 26/100 [00:55<02:38,  2.15s/it]

Ending


In [22]:
# to csv file
Cinema_CSV = pd.DataFrame(cinema)
Cinema_CSV.to_csv("cinema.csv",index=False,encoding="utf_8_sig")

In [23]:
# to jason file, cuz another ipynb will used
cinema_jason = {'cinemaId':cinema['cinemaId'],'cinemaName':cinema['cinemaName']} # for other jupyter load
with open("cinema.json","wt") as f:
    json.dump(cinema_jason,f)    

In [24]:
# excel preview
Cinema_CSV

,cinemaId,cinemaAddress,cinemaName,cinemaTele,cinemaImg,cinemaBusTwo,cinemaInfo,cinemaGoogle
0,A,台北市信義區松壽路20號,台北信義威秀影城,(02) 8780-5566,C:\Users\User\Desktop\Cinema_Img\_台北信義威秀影城.jpg,捷運：市政府站3號出口、捷運象山站 1號出口、台北101/世貿站4號出口\n公車︰可搭乘 2...,台北信義威秀影城內有時髦前衛、聲光炫麗的裝潢，為全國第一座專門為放映電影而量身打造的建築設計...,https://www.google.com/maps?t=m&hl=zh-TW&gl=TW...
1,B,台北市大同區市民大道一段209號5樓,台北京站威秀影城,(02) 2552-5511,C:\Users\User\Desktop\Cinema_Img\_台北京站威秀影城.jpg,捷運/高鐵/台鐵：台北車站站，往中山地下街方向至R1出口，或經台北地下街Y1、Y3、Y5出口...,台北京站威秀影城為威秀影城全省第八家影城，也是正式更名為威秀影城後，第一家全新開幕的影城。座...,https://www.google.com/maps?t=m&hl=zh-TW&gl=TW...
2,D,新北市板橋區新站路28號10樓,板橋大遠百威秀影城,(02) 7738-6608,C:\Users\User\Desktop\Cinema_Img\_板橋大遠百威秀影城.jpg,捷運/高鐵/台鐵：板橋站3 A 出口，往台鐵1F東2門方向右轉步行約3~5分鐘，或由台鐵B1...,板橋大遠百威秀影城是威秀影城於新北市的第一個據點。座落於新板特區板橋大遠百複合式商場內的10...,https://www.google.com/maps?t=m&hl=zh-TW&gl=TW...
3,E,新北市林口區文化三路一段356號3樓,林口威秀影城,(02) 2606-8099,C:\Users\User\Desktop\Cinema_Img\_林口威秀影城.jpg,公車：1)台北前往可搭乘：786、858、898、920、925、936、937、945、9...,林口MITSUI OUTLET PARK威秀影城，為威秀影城以開創性概念打造的全新據點。\r...,https://www.google.com/maps?t=m&hl=zh-TW&gl=TW...
4,G,新竹市西大路323號8樓,新竹大遠百威秀影城,(03) 524-1166,C:\Users\User\Desktop\Cinema_Img\_新竹大遠百威秀影城.jpg,開車：新竹交流道→光復路往市區方向→食品路左轉→西大路右轉即可到達公車︰可搭乘 5號公車至「...,新竹大遠百威秀影城位於新竹市西大路大遠百娛樂城8樓及10樓，引進世界級的影音設備，同步上映全...,https://www.google.com/maps?t=m&hl=zh-TW&gl=TW...
5,I,新竹市民權路176號4樓之3,新竹巨城威秀影城,(03) 534-6999,C:\Users\User\Desktop\Cinema_Img\_新竹巨城威秀影城.jpg,開車：新竹交流道→光復路往市區方向→民族路右轉→中央路右轉即可到達公車︰可搭乘 52號或53...,新竹巨城威秀影城位於新竹最大的百貨購物中心，這個以「家」的視覺風格為主要設計，採用木質色系裝...,https://www.google.com/maps?t=m&hl=zh-TW&gl=TW...
6,J,苗栗縣頭份市中央路105號7樓,頭份尚順威秀影城,(037) 686-866,C:\Users\User\Desktop\Cinema_Img\_頭份尚順威秀影城.jpg,開車：國道一號南上或北下，由頭份交流道下車程約5分鐘即可抵達台鐵：由台鐵竹南站下車後搭乘苗栗...,苗栗尚順威秀影城，讓座落在山中之城的這塊土地擁有電影養份；本影城呼應素有山城之稱的苗栗，打造...,https://www.google.com/maps?t=m&hl=zh-TW&gl=TW...
7,K,台中市西屯區台灣大道三段251號13樓,台中大遠百威秀影城,(04) 2258-8511,C:\Users\User\Desktop\Cinema_Img\_台中大遠百威秀影城.jpg,開車：1)中山高速公路台中交流道(台中、沙鹿))下往台中市方向2)中彰快速公路市政路出口下交...,台中大遠百威秀影城為2011年底於台中增設的第三個據點，這座中台灣最新的複合式影城以原野大地...,https://www.google.com/maps?t=m&hl=zh-TW&gl=TW...
8,L,台中市西屯區河南路三段120-1號4樓,台中威秀影城,(04) 3606-5566,C:\Users\User\Desktop\Cinema_Img\_台中威秀影城.jpg,公車︰《高鐵台中站前往》1.轉乘往「台中市區」路線，至「秋紅谷」站下車，步行約5分鐘。2.豐...,TIGER CITY威秀影城於2011年11月25日全新改裝開幕，眾所矚目的大台中首座商用數...,https://www.google.com/maps?t=m&hl=zh-TW&gl=TW...
9,O,台南市公園路60號5樓,台南大遠百威秀影城,(06) 600-5566,C:\Users\User\Desktop\Cinema_Img\_台南大遠百威秀影城.jpg,公車：可搭乘台南市公車至「台南大遠百」站下車,威秀影城於2002年7月5日正式在府城台南開幕，這座在台南核心地帶的威秀影城始終是年輕學生族...,https://www.google.com/maps?t=m&hl=zh-TW&gl=TW...
